In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QTableWidget, QTableWidgetItem, QVBoxLayout, QWidget, QLineEdit, QHBoxLayout
import mysql.connector

class MaFenetre(QMainWindow):
    def __init__(self):
        super().__init__()

        # Informations de connexion à MySQL
        self.host = "192.168.20.61"
        self.user = "ludo"
        self.password = "root"
        self.database = "batiment_intelligent"

        # Connexion à la base de données MySQL au démarrage de l'application
        self.connexion_db = self.connect_to_database()

        # Layout principal
        layout_principal = QVBoxLayout()

        # Layout pour les boutons et le champ de texte
        layout_boutons = QHBoxLayout()

        # Bouton pour afficher les tables
        bouton_afficher_tables = QPushButton('Afficher les tables', self)
        bouton_afficher_tables.clicked.connect(self.clique_sur_bouton_afficher_tables)
        layout_boutons.addWidget(bouton_afficher_tables)

        # Champ de texte pour le nom de la table à créer
        self.champ_nom_table_creer = QLineEdit(self)
        layout_boutons.addWidget(self.champ_nom_table_creer)

        # Bouton pour créer une table
        bouton_creer_table = QPushButton('Créer une table', self)
        bouton_creer_table.clicked.connect(self.clique_sur_bouton_creer_table)
        layout_boutons.addWidget(bouton_creer_table)

        layout_principal.addLayout(layout_boutons)

        # Tableau pour afficher les données de la table "capteurs"
        self.table_capteurs = QTableWidget(self)
        layout_principal.addWidget(self.table_capteurs)

        # Ajout des widgets au widget central
        widget_central = QWidget()
        widget_central.setLayout(layout_principal)
        self.setCentralWidget(widget_central)

    def connect_to_database(self):
        try:
            # Connexion à la base de données MySQL
            connection = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            return connection
        except Exception as e:
            print(f'Erreur de connexion à la base de données : {str(e)}')
            return None

    def clique_sur_bouton_afficher_tables(self):
        try:
            # Afficher les données de la table "capteurs"
            self.afficher_table("capteurs", self.table_capteurs)
        except Exception as e:
            print(f'Erreur lors de l\'affichage des tables : {str(e)}')

    def afficher_table(self, nom_table, table_widget):
        try:
            cursor = self.connexion_db.cursor()
            cursor.execute(f'SELECT * FROM {nom_table}')
            resultats = cursor.fetchall()

            # Obtenir les noms des colonnes
            colonnes = [description[0] for description in cursor.description]

            # Configuration de la table widget
            table_widget.setRowCount(len(resultats))
            table_widget.setColumnCount(len(colonnes))
            table_widget.setHorizontalHeaderLabels(colonnes)

            # Remplir le tableau avec les données
            for i, ligne in enumerate(resultats):
                for j, valeur in enumerate(ligne):
                    item = QTableWidgetItem(str(valeur))
                    table_widget.setItem(i, j, item)
        except Exception as e:
            print(f'Erreur lors de l\'affichage de la table : {str(e)}')

    def clique_sur_bouton_creer_table(self):
        try:
            # Récupérer le nom de la table à créer depuis le champ de texte
            nom_table = self.champ_nom_table_creer.text()

            # Créer la table avec le nom spécifié
            if nom_table:
                cursor = self.connexion_db.cursor()
                cursor.execute(f'CREATE TABLE IF NOT EXISTS {nom_table} (id INT AUTO_INCREMENT PRIMARY KEY, nom VARCHAR(255))')
                self.connexion_db.commit()

                print(f'Table "{nom_table}" créée avec succès!')
            else:
                print('Veuillez spécifier un nom de table.')
        except Exception as e:
            print(f'Erreur lors de la création de la table : {str(e)}')

if __name__ == '__main__':
    app = QApplication(sys.argv)
    fenetre = MaFenetre()
    fenetre.show()
    sys.exit(app.exec_())

Table "pastaga" créée avec succès!
